# UrbanSim Templates demo

Sam Maurer, Feb 2021

### Background

[UrbanSim](https://github.com/udst/urbansim) is a platform for modeling land use in cities. It runs in Python and uses the [Orca](https://github.com/udst/orca) task orchestration system. 

Orca breaks a model into "steps", Python functions that can be assembled on the fly into linear or cyclical pipelines. Orca is designed for workflows like city simulation where the data representing a model's state is so large that it needs to be managed outside the task graph. Steps refer to tables and columns of data by name rather than passing the data directly.

UrbanSim [Templates](https://github.com/udst/urbansim_templates) is a library that provides automated building blocks for Orca-based models. The templates were developed to reduce the need for custom code and improve the portability of model steps.

Currently we have templates for (a) regression, (b) binary Logit, (c) multinomial Logit estimated with [PyLogit](https://github.com/timothyb0912/pylogit) (best choice for flexible utility expressions), and (d) multinomial Logit estimated with [ChoiceModels](https://github.com/udst/choicemodels) (best choice for sampling of interchangeable alternatives).

### Documentation

Full UrbanSim Templates documentation: https://udst.github.io/urbansim_templates/

### Installation

You can install `orca` and `urbansim_templates` from Pip or Conda Forge.

In [1]:
import pandas as pd

print(pd.__version__)

1.2.1


In [2]:
import orca

print(orca.__version__)

1.5.4


In [3]:
import urbansim_templates

print(urbansim_templates.__version__)

0.2.dev9


In [4]:
# Making the notebook output clearer
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Setting up ModelManager

[ModelManager](https://udst.github.io/urbansim_templates/modelmanager.html) is part of the Templates library. It's an extension to Orca for saving and loading template-based model steps. 

By default it will look for a folder named `configs` in your current working directory, where it will read and save yaml representations of model steps. If there are already model steps there, the corresponding template classes need to be loaded before initializing ModelManager.

In [5]:
from urbansim_templates.models import OLSRegressionStep
from urbansim_templates import modelmanager

modelmanager.initialize()

No yaml files found in path 'configs'


### Setting up data

We'll load a DataFrame and register it with Orca, so that our statistical models can refer to it.

In [6]:
df = pd.read_csv('data/buildings-demo.csv').dropna()
len(df)

482

In [7]:
orca.add_table('buildings', df)

In [8]:
orca.get_table('buildings').to_frame().head()

,building_id,parcel_id,development_type_id,improvement_value,residential_units,residential_sqft,sqft_per_unit,non_residential_sqft,building_sqft,res_price_per_sqft,stories,year_built,sale_price,sale_year,building_type_id
3,7,328712,1,0.0,1,1393,1393.0,0,0.0,0.000000,1,2008,670250.0,2008.0,1
4,9,742661,1,116580.0,1,1018,1018.0,0,1018.0,474.350534,1,1946,703000.0,2007.0,1
6,11,716626,1,457526.0,1,3693,3693.0,0,3693.0,124.824432,1,1998,95000.0,1996.0,1
10,15,742822,1,95050.0,1,1106,1106.0,0,1106.0,448.074261,1,1957,675000.0,2005.0,1
13,18,743444,1,166000.0,1,1354,1354.0,0,1354.0,411.506401,1,1951,18500.0,2006.0,1


### Creating a model step

Now we can choose a [template](https://udst.github.io/urbansim_templates/model-steps.html) and use it to fit a model.

In [9]:
from urbansim_templates.models import OLSRegressionStep

m = OLSRegressionStep()
m.name = 'price-prediction'
m.tables = 'buildings'
m.model_expression = 'np.log1p(res_price_per_sqft) ~ non_residential_sqft>0 + year_built<1960'

In [10]:
m.fit()

                                 OLS Regression Results                                 
Dep. Variable:     np.log1p(res_price_per_sqft)   R-squared:                       0.398
Model:                                      OLS   Adj. R-squared:                  0.395
Method:                           Least Squares   F-statistic:                     158.1
Date:                          Tue, 09 Feb 2021   Prob (F-statistic):           1.93e-53
Time:                                  12:02:09   Log-Likelihood:                -598.98
No. Observations:                           482   AIC:                             1204.
Df Residuals:                               479   BIC:                             1216.
Df Model:                                     2                                         
Covariance Type:                      nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
---------

### Registering the step

Now we can "register" the step with ModelManager. This saves a copy to disk (in the `configs` folder), and passes a copy to Orca so it can be run as part of a sequence of other steps for validation or simulation.

In [11]:
modelmanager.register(m)

Saving 'price-prediction.yaml': /Users/maurer/Dropbox/Git-imac/udst/urbansim_templates/examples/configs
Registering model step 'price-prediction'


### Making changes

Previously registered steps can be retrieved, modified, and re-registered as needed.

In [12]:
modelmanager.list_steps()

[{'name': 'price-prediction', 'template': 'OLSRegressionStep', 'tags': []}]

In [13]:
m2 = modelmanager.get_step('price-prediction')

In [14]:
m2.name = 'better-price-prediction'
# here you can edit the specification and re-fit, etc.

modelmanager.register(m2)

Saving 'better-price-prediction.yaml': /Users/maurer/Dropbox/Git-imac/udst/urbansim_templates/examples/configs
Registering model step 'better-price-prediction'


In [15]:
modelmanager.remove_step('better-price-prediction')

Removing 'better-price-prediction' and 'better-price-prediction.yaml'
